In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au


In [2]:
input_text = au.read_txt_file_lines('input.txt')

In [3]:
#  |  vertical 
#  _  horizontal
#  L  NE
#  J  NW
#  7  SW
#  F  SE 
#  .  nothing
#  S  start

## from seein ght einput, we can see that the start is NW (J)
# start_pipe = 'J'

In [30]:
dict_directions = {
    ## (pipe, direction) : new direction
    ('L', 'S'): 'E',
    ('L', 'W'): 'N',    
    ('J', 'S'): 'W',
    ('J', 'E'): 'N',
    ('7', 'N'): 'W',
    ('7', 'E'): 'S',
    ('F', 'N'): 'E',
    ('F', 'W'): 'S',  
    ('|', 'N'): 'N',
    ('|', 'S'): 'S',
    ('-', 'E'): 'E',
    ('-', 'W'): 'W'
}

def traject_pipe(input_text, dict_directions=dict_directions, new_input=None, new_input_char='pipe',
                start_dir='N'):
    ## find start pos:
    for row in range(len(input_text)):
        if 'S' in input_text[row]:
            start_pos = (row, input_text[row].index('S'))
            break
     
    n_steps = 0 
    prev_pointer = ''
    curr_dir = start_dir
    curr_pos = start_pos
    print(f'starting at {curr_pos}')
    continue_traj = True
    while continue_traj:
        n_steps += 1
        curr_row, curr_col = curr_pos
        assert curr_col >= 0 and curr_row >= 0, 'out of bounds'
        assert curr_col < len(input_text[0]) and curr_row < len(input_text), 'out of bounds'
        
        ## calculate next position
        if curr_dir == 'N':
            next_pos = (curr_row - 1, curr_col)
        elif curr_dir == 'E':
            next_pos = (curr_row, curr_col + 1)
        elif curr_dir == 'S':
            next_pos = (curr_row + 1, curr_col)
        elif curr_dir == 'W':
            next_pos = (curr_row, curr_col - 1)

        ## find new pipe:
        next_pipe = input_text[next_pos[0]][next_pos[1]]

        ## option to create new map (new_input)
        if new_input is not None:
            if new_input_char == 'pipe':  # map pipe piece
                new_input[curr_row][curr_col] = input_text[curr_row][curr_col]
            elif new_input_char == 'direction':  # map current direction
                new_input[curr_row][curr_col] = curr_dir
            elif new_input_char == 'inside_pointer_90deg_left':  ## map inside pointer 90 deg left of curr_dir
                ### we know that start dir is N and start pointer is W. I believe that inside_pointer always pointers 90 deg left of curr_dir
                if curr_dir == 'N':
                    new_pointer = 'W'
                elif curr_dir == 'E':
                    new_pointer = 'N'
                elif curr_dir == 'S':
                    new_pointer = 'E'
                elif curr_dir == 'W':
                    new_pointer = 'S'
                new_input[curr_row][curr_col] = new_pointer + prev_pointer
                prev_pointer = new_pointer
            elif new_input_char == 'inside_pointer_90deg_right':  # map inside pointer 90 deg right of curr_dir
                if curr_dir == 'N':
                    new_pointer = 'E'
                elif curr_dir == 'E':
                    new_pointer = 'S'
                elif curr_dir == 'S':
                    new_pointer = 'W'
                elif curr_dir == 'W':
                    new_pointer = 'N'
                new_input[curr_row][curr_col] = new_pointer + prev_pointer
                prev_pointer = new_pointer
            elif new_input_char == 'X':  # map X
                new_input[curr_row][curr_col] = 'X'

        ## check end:
        if next_pipe == 'S':
            continue_traj = False
            break
        elif next_pipe == '.':
            assert False, f'found . at {next_pos}'
        else:
            key = (next_pipe, curr_dir)
            assert key in dict_directions, f'key {key} not in dict_directions at {next_pos}, {n_steps} steps'
            new_dir = dict_directions[key]
            curr_dir = new_dir
            curr_pos = next_pos 

    return n_steps

n_steps_pipe = traject_pipe(input_text=input_text, start_dir='W')
print(f'part 1: {n_steps_pipe // 2}')

starting at (76, 109)
part 1: 6697


## part 2
Plan:
- First, change all pipes that arent part of the main loop into `.`
- Then, traject through pipe, with vector pointing inside. Map the vector in a new map `new_input`. 
- **Update:** NB: on corners, there can be two vectors. So I now map multiple directions. (Without these I got 416 one way and 421 other way)
- Then go through map of vectors. If identify a `.`, see if it is next to a vector pointing to it or and `X`. If so, change to `X`. (This recursively captures all `X`).

In [25]:
new_input = [] 
for row in range(len(input_text)):
    new_input.append(['.'] * len(input_text[0]))

## only difference it's now a list of ['.', '.'] instead of ['..'] to make it mutable. 
n_steps_pipe = traject_pipe(input_text=input_text, new_input=new_input, new_input_char='inside_pointer_90deg_left', start_dir='W')

## Visualise new map:
# new_input_form = [''.join(row) for row in new_input]
# new_input_form

starting at (76, 109)


In [31]:
n_r = len(input_text)
n_c = len(input_text[0])

def check_if_dot_inside(map_inside_pointers, row, col):
    '''see if dot is next to a X or a vector pointing to it.'''
    if map_inside_pointers[row][col] != '.':
        return False
    if col > 0 and np.any([pointer in ['E', 'X'] for pointer in map_inside_pointers[row][col - 1]]):  # use any constructions because it can contain multiple vectors (eg ['NW', 'SE', 'S', 'S', .. ])
        return True
    elif col < n_c - 1 and np.any([pointer in ['W', 'X'] for pointer in map_inside_pointers[row][col + 1]]):
        return True
    elif row > 0 and np.any([pointer in ['S', 'X'] for pointer in map_inside_pointers[row - 1][col]]):
        return True
    elif row < n_r - 1 and np.any([pointer in ['N', 'X'] for pointer in map_inside_pointers[row + 1][col]]):
        return True
    else:
        return False

def mark_dots_inside(map_inside_pointers, row, col):
    '''Check if dot should be marked X. If so, check all its neighbours recursively.'''
    found_X = check_if_dot_inside(map_inside_pointers, row, col)
    if found_X:
        map_inside_pointers[row][col] = 'X'
        mark_dots_inside(map_inside_pointers, row, col - 1)
        mark_dots_inside(map_inside_pointers, row, col + 1)
        mark_dots_inside(map_inside_pointers, row - 1, col)
        mark_dots_inside(map_inside_pointers, row + 1, col)
    return None

## Loop through all elements of vector map. Check if it's a dot, and if so, if it should be marked X based on neighbouring vectors and X's..
map_use = new_input
for row in range(n_r):
    for col in range(n_c):
        if map_use[row][col] == '.':
            mark_dots_inside(map_use, row, col)

n_X = sum([sum(['X' in el for el in row]) for row in map_use])
print(f'part 2: {n_X}')

part 2: 423
